In [ ]:
#!/usr/bin/env python
# -*- encoding:utf-8-*-
import firebase_admin                   # for firebase
from firebase_admin import credentials  # for firebase
from firebase_admin import db           # for firebase 
import cv2                              # for face recognition
import numpy as np                      # for face recognition
import os                               # for face recognition
import serial                           # for Serial communication
import time                             # for delay function

# face recognition
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascades/haarcascade_frontalface_default.xml"
fist_cascade = cv2.CascadeClassifier('haarcascades/hand.xml')
palm_cascade = cv2.CascadeClassifier('haarcascades/palm.xml')
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0
mode_id = 1
count = 0
mode_ar = ['hand wash', 'face wash', 'hand']
ok_tmp = 0

# names related to ids: example ==> loze: id=1,  etc
names = ['None', 'Heesang', 'Nayoung', 'Jeonghoon']

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 800) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

"""
input
    none
output
    global id
    global mode_id
    global ok_tmp
    return k
- read image taken by pi cam
- flip the img and change to gray color for img recognition
- indicate face, fist and palm
- find an user_no from the indicated face
- when fist is indicated, increase count and change mode
- when palm is indicated, select the mode and make ok_tmp to 1
- indicate keyboardinturrupt and return k
"""
def face_recognition():
    ret, img =cam.read()
    img = cv2.flip(img, 1)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(3,3),0)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH))
       )

    fists = fist_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize = (120,120)
    )
    
    palms = palm_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize = (120,120)
    )    
    
    global id
    global mode_id
    global ok_tmp
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        # Check if confidence is less them 100 ==> "0" is perfect match
        if (confidence < 70):
            name = names[id]
        else:
            name = "guest"
            id = 0
        
        cv2.putText(img, str(name), (x+5,y-5), font, 1, (255,255,255), 2)

    for (x,y,w,h) in fists:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
        global count
        count = count + 1
        
        if (count<= 10):
            mode_id = 3
            cv2.putText(img, mode_ar[0], (x+5,y-5), font, 1, (255,255,255), 2)
        elif (count<= 20):
            mode_id = 2
            cv2.putText(img, mode_ar[1], (x+5,y-5), font, 1, (255,255,255), 2)
        elif (count<= 30):
            mode_id = 1
            cv2.putText(img, mode_ar[2], (x+5,y-5), font, 1, (255,255,255), 2)
        else:
            count = 0

    for (x,y,w,h) in palms:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
        cv2.putText(img, "OK", (x+5,y-5), font, 1, (255,255,255), 2)
        ok_tmp = 1
        
        
    cv2.imshow('camera',img)
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    
    return k

#credit firebase by json key
cred = credentials.Certificate('#json key file path')
firebase_admin.initialize_app(cred, {'databaseURL' : '#firebase db url'})

"""
input
    userno(str) from face_recognition
    mode(int) from face_recognition
output : return de_temp, face_after, face_before, face_clean, face_temp, 
        hand_after, hand_before, hand_clean, hand_temp, uv, sewer, sewer_now  (all str)
        values get from firebase in real time
- get db values
- assign each values to variables according to key
- assign only value part(exclude \ or units)        
"""
def get_from_db(userno, mode): 
    dir = 'user/user' + userno

    start = db.reference(dir)
    info = start.get()
    for key, value in info.items():
        # use only 2 digit after floating point
        if(key == 'de_temp'):
            de_temp = value
            de_temp = de_temp[0:5]
        # exclude \ and units 's'
        elif(key == 'face_after'):
            face_after = value
            face_after = face_after[1: len(face_after)-2]
        elif(key == 'face_before'):
            face_before = value
            face_before = face_before[1: len(face_before)-2]
        elif(key == 'face_clean'):
            face_clean = value
            face_clean = face_clean[1: len(face_clean)-2]
        # use only 2 digit after floating point
        elif(key == 'face_temp'):
            face_temp = value
            face_temp = face_temp[0:5]
         # exclude \ and units 's'
        elif(key == 'hand_after'):
            hand_after = value
            hand_after = hand_after[1: len(hand_after)-2]
        elif(key == 'hand_before'):
            hand_before = value
            hand_before = hand_before[1: len(hand_after)-2]
        elif(key == 'hand_clean'):
            hand_clean = value
            hand_clean = hand_clean[1: len(hand_after)-2]
        # use only 2 digit after floating point
        elif(key == 'hand_temp'):
            hand_temp = value
            hand_temp = hand_temp[0:5]
            
    dir = 'others'
    start = db.reference(dir)
    info = start.get()
    
    
    for key, value in info.items():
         # exclude \ and units 'h'
        if(key == 'UV_cycle'):
            uv = value
            uv = uv[1: len(uv) - 2]
         # exclude \ and units 'week'
        elif(key == 'sewer_cycle'):
            sewer = value
            sewer = sewer[1: len(sewer) - 5]
        elif(key == 'sewer_now'):
            sewer_now = value
            
    return de_temp, face_after, face_before, face_clean, face_temp, hand_after, hand_before, hand_clean, hand_temp, uv, sewer, sewer_now

"""
input
    mode(int), 
    uv, de_temp, face_after, face_before, face_clean, hand_after, hand_before, hand_clean, sewer(all str)
output
    global output(str)
- set digit to send the values to arduino in one string
- mode ok before clean after (3 digit) sewer cycle(1 digit) sewer_now led_cycle(3 digit) water_temp(4 digit)
- mode1 000 2 face 3 hand 
"""
def make_digit(mode, uv, de_temp, face_temp, hand_temp, face_after, face_before, face_clean, hand_after, hand_before, hand_clean, sewer):
    global output
    
    # to add to output(str)
    mode = str(mode)

    # set temp to have 2 digit after floaing point
    if(len(de_temp) == 2):
        de_temp = de_temp + '000'
    elif(len(de_temp) == 4):
        de_temp = de_temp + '0'
        
    if(len(face_temp) == 2):
        face_temp = face_temp + '000'
    elif(len(hand_temp) == 4):
        hand_temp = hand_temp + '0'

    if(len(de_temp) == 2):
        de_temp = de_temp + '000'
    elif(len(de_temp) == 4):
        de_temp = de_temp + '0'
        
    # set to 3 digit
    if(len(uv) == 1):
        uv = '00' + uv
    elif(len(uv) == 2):
        uv = '0' + uv
        
    if(len(face_before) == 1):
        face_before = '00' + face_before
    elif(len(face_before) == 2):
        face_before = '0' + face_before

    if(len(face_after) == 1):
        face_after = '00' + face_after
    elif(len(face_after) == 2):
        face_after = '0' + face_after

    if(len(face_clean) == 1):
        face_clean = '00' + face_clean
    elif(len(face_clean) == 2):
        face_clean = '0' + face_clean

    if(len(hand_after) == 1):
        hand_after = '00' + hand_after 
    elif(len(hand_after) == 2):
        hand_after = '0' + hand_after

    if(len(hand_before) == 1):
        hand_before = '00' + hand_before
    elif(len(hand_before) == 2):
        hand_before = '0' + hand_before

    if(len(hand_clean) == 1):
        hand_clean = '00' + hand_clean
    elif(len(hand_clean) == 2):
        hand_clean = '0' + hand_clean

    # if mode = 2, put face values, mode = 3, hand values, otherwise, default values
    if (mode == '2'):
        output = '2' + ok + face_before + face_clean + face_after + sewer + uv + face_temp[0:2] + face_temp[3:5]
    elif(mode == '3'):
        output = '3' + ok + hand_before + hand_clean + hand_after + sewer + uv + hand_temp[0:2] + hand_temp[3:5]
    else :
        output = '1' + ok + '000000000' + sewer + uv + de_temp[0:2] + de_temp[3:5]
        
    return output

"""
input
    remains(str) : from arduino through serial communication
output
    retrun f_face_remain, f_hand_remain, f_sewer_now
- assign remain values to each variables
"""
def get_from_arduino(remains):
    f_face_remain = remains[1]
    f_hand_remain = remains[2]
    f_sewer_remain = remains[3]
    f_sewer_now = remains[4]
    
    return f_face_remain, f_hand_remain, f_sewer_remain, f_sewer_now    

"""
input
    face_remain, hand_remain, sewer_remain : if reamins are under limit, 1
    sewer_now : if now sewering, 1
output
    update firebase db
"""
def update_db(face_remain, hand_remain, sewer_remain, sewer_now):
    ref = db.reference('others')
    ref.update({
        'face_remain' : face_remain,
        'hand_remain' : hand_remain,
        'sewer_remain' : sewer_remain,
        'sewer_now' : sewer_now
    })

"""
input
    global remains : from arduino
output
    none
- top function of firebase_to arduino
"""
def firebase_to_arduino():
    global remains
    de_temp, face_after, face_before, face_clean, face_temp, hand_after, hand_before, hand_clean, hand_temp, uv, sewer, sewer_now = get_from_db(userno ,mode)
    output = make_digit(mode, uv, de_temp, face_temp, hand_temp, face_after, face_before, face_clean, hand_after, hand_before, hand_clean, sewer)
    face_remain, hand_remain, sewer_remain, sewer_now = get_from_arduino(remains)
    update_db(face_remain, hand_remain, sewer_remain, sewer_now)

# serial port connect
ser1 = serial.Serial('COM6', 9600)      # COM6 port connect , Mega
time.sleep(2)                           # wait for 2 sec

"""
input
    data_info(str) : from rpi(get from firebase in real time)
output
    send info_operate throuth serial communication to arduino
- send str from python to arduino
"""
def transData(data_info):
    if ser1.readable():
        info_operate = str(data_info)
        info_operate = info_operate.encode('utf-8')
        ser1.write(info_operate)
        time.sleep(1)
        
# arduino -> python
"""
input
    data from arduino throuth serial communication
output
    return made_alarm(str)
- send str from python to arduino
- get remain values and sewering info from arduino
"""
def getMessage():
    if ser1.readable():
        made_alarm = ser1.readline()                
        made_alarm = made_alarm.decode() 
        return made_alarm

remains = "88888"

"""
total while loop
- for real time updating
- get user's no, mode's no and ok from face_recognition()
- get output from firebase_to_arduino() using user_no, ok, mode
- send output to arduino
- get remain values from arduino
- update remain values to firebase
"""
while(1):
    face_recognition()
    
    userno = id
    mode = mode_id
    userno = str(userno)
    ok = str(ok_tmp)
    output = str()
    firebase_to_arduino()
    
    if ser1.readable():
        data_tmp = output
        data_info = int('0' + data_tmp)       # str to int
    # rpi to arduino
    transData(data_info)        
    # arduino to rpi
    remain_get= getMessage()   
    remains = remain_get

# clean up for cam, cv
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()